In [ ]:
import sys
import os
import logging
import pandas as pd
import datasets
from pprint import pprint
KEY = '2-NOTEBOOK'
WORKSPACE_PATH = os.getcwd().split(KEY)[0]
print(WORKSPACE_PATH); os.chdir(WORKSPACE_PATH)
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

SPACE = {
    'DATA_RAW': f'_Data/0-Data_Raw',
    'DATA_RFT': f'_Data/1-Data_RFT',
    'DATA_CASE': f'_Data/2-Data_CASE',
    'DATA_AIDATA': f'_Data/3-Data_AIDATA',
    'DATA_EXTERNAL': f'code/external',
    'DATA_HFDATA': f'_Data/5-Data_HFData',
    'CODE_FN': f'code/pipeline',
    'MODEL_ROOT': f'./_Model',
}
assert os.path.exists(SPACE['CODE_FN']), f'{SPACE["CODE_FN"]} not found'
print(SPACE['CODE_FN'])
sys.path.append(SPACE['CODE_FN'])

os.environ["CUDA_VISIBLE_DEVICES"]="0"

# AIData - MaskedLM

In [ ]:
from recfldtkn.aidata_base.entry import EntryAIData_Builder

OneAIDataName = 'PreTrainBench'
Task = 'CausalLM'
Version = 'v0515'
DataName = f'{OneAIDataName}-{Task}-{Version}'

CF_DataName = 'PreTrainBench-CGM5MinEntry-1ea9d787eef20fb7'
CohortName_list = ['WellDoc2022CGM', 'WellDoc2025ALS', 'WellDoc2025CVS', 'WellDoc2025LLY']
CF_DataName_list = [f'{i}/{CF_DataName}' for i in CohortName_list]
entry = EntryAIData_Builder(SPACE=SPACE)
dataset = entry.merge_one_cf_dataset(CF_DataName_list)

In [ ]:
dataset

In [ ]:
data_config = dataset.info.config_name 
CF_to_CFvocab = data_config['CF_to_CFvocab']

CFName = 'HM5MinStep'
interval_delta = pd.Timedelta(minutes=5)
idx2tkn = [pd.Timestamp('2022-01-01 00:00:00') + interval_delta * i for i in range(24 * 12)]
idx2tkn = [f'{i.hour:02d}:{i.minute:02d}' for i in idx2tkn]
tkn2idx = {tkn: idx for idx, tkn in enumerate(idx2tkn)}
CF_to_CFvocab = data_config['CF_to_CFvocab']
CF_to_CFvocab[CFName] = {'idx2tkn': idx2tkn, 'tkn2idx': tkn2idx}

CFName = 'CGMValue'
idx2tkn = ["PAD", "UNKNOWN", "MASK"] + [f'Other_{i}' for i in range(0, 7)] + [str(i) for i in range(10, 401)]
tkn2idx = {tkn: idx for idx, tkn in enumerate(idx2tkn)}
CF_to_CFvocab[CFName] = {'idx2tkn': idx2tkn, 'tkn2idx': tkn2idx}

print([i for i in CF_to_CFvocab])

In [ ]:
OneEntryArgs = {
    # ----------------- Input Part -----------------
    'Split_Part': {}, # set to empty dict to avoid split
    'Input_Part': {
        'EntryInputMethod': '1TknInStepWt5MinHM',
        'CF_list': [
            'CGMValueBf24h',
            'CGMValueAf2h',
            'CGMValueAf2to8h',
        ],
        'BeforePeriods': ['Bf24h'],
        'AfterPeriods': ['Af2h', 'Af2to8h'],
        'TimeIndex': True, 
        'InferenceMode': False, # True, # True, # False, # True, 
        'TargetField': 'CGMValue',
        'TargetRange': [40, 400],
        # 'HM': None, 
        'HM': {'start': -24, 'unit': 'h', 'interval': '5m'},
    }, 
    'Output_Part': {
        'EntryOutputMethod': 'CausalLM',
        'selected_columns': ['input_ids', 'hm_ids', 'labels', 'split_timebin', 'stratum'],
        'set_transform': False,
        'num_proc': 12, 
    },
}

entry = EntryAIData_Builder(OneEntryArgs=OneEntryArgs, SPACE=SPACE)
entry.CF_to_CFvocab = CF_to_CFvocab

In [ ]:
ds = dataset.select(range(10))
Data = {'ds_case': ds}
Data = entry.setup_EntryFn_to_Data(Data, CF_to_CFvocab)
df = Data['ds_tfm'].to_pandas()
df.iloc[1]

In [ ]:
ds = dataset#.select(range(10))
Data = {'ds_case': ds}
Data = entry.setup_EntryFn_to_Data(Data, CF_to_CFvocab)

In [ ]:
path = os.path.join(SPACE['DATA_HFDATA'], f'{DataName}')
print(path)
ds_tfm = Data['ds_tfm']
ds_tfm.save_to_disk(path)